# Solving Cart-Pole with SVP

This notebook shows how to use the proposed structured value-based planning (SVP) approach to generate the state-action $Q$-value function for the classic cart-pole problem. The correctness of the solution is verified by trajectory simulations as well as the final metrics comparisons.

## Problem definition

We here focus on the cart-pole problem, which has a higher state space dimension, i.e., 4 dims.

The cart-pole problem consists in balancing in upright position a pole attached to a cart moving on a frictionless track. The cart can be controlled by means of a limited force within 10$N$ that is possible to apply both to the left or to the right of the cart. The goal is to keep the pole on the upright equilibrium position.
The physical dynamics of the system is described by the angle and the angular speed of the pole, and the position and the speed of the cart, i.e., $(\theta, \dot{\theta}, x, \dot{x})$. Denote $\tau$ as the time interval between decisions, $u$ as the force input on the cart, the dynamics can be written as

\begin{align}
    & \ddot{\theta} := \frac{g\sin{\theta} - \frac{u+ml \dot{\theta}^2 \sin{\theta}}{m_c+m} \cos{\theta}}{l\left( \frac{4}{3} - \frac{m\cos^2{\theta}}{m_c+m} \right)},\\
    & \ddot{x} := \frac{u + ml\left( \dot{\theta}^2 \sin{\theta} - \ddot{\theta}\cos{\theta}\right) }{m_c+m},\\
    & \theta := \theta + \dot{\theta}~\tau,\\
    & \dot{\theta} := \dot{\theta} + \ddot{\theta}~\tau,\\
    & x := x + \dot{x}~\tau,\\
    & \dot{x} := \dot{x} + \ddot{x}~\tau,\\
\end{align}

where $g=9.8m/s^2$ corresponds to the gravity acceleration, $m_c = 1kg$ to the mass of the cart, $m = 0.1kg$ to the mass of the pole, $l = 0.5m$ to half of the pole length, and $u$ corresponds to the force applied to the cart.

A reward function that favors the pole in an upright position, i.e., characterized by keeping the pole in vertical position between $\left|\theta\right| \le \frac{12\pi}{180}$, is expressed as

\begin{equation}
    r(\theta) = \cos^4{(15 \theta)}.
\end{equation}

In the simulation, the state space is $[-\frac{\pi}{2}, \frac{\pi}{2}]$ for $\theta$, $[-3,3]$ for $\dot{\theta}$, $[-2.4,2.4]$ for $x$ and $[-3.5,3.5]$ for $\dot{x}$. We limit the input force in $[-10,10]$ and set $\tau=0.1$.
We discretize each dimension of the state space into 10 values, and action space into 1000 values, which forms an $Q$-value function  a matrix of dimension $10000\times 1000$.

## Structured Value-based Planning (SVP)

The proposed structured value-based planning (SVP) approach is based on the $Q$-value iteration. At the $t$-th iteration, instead of a full pass over all state-action pairs:
- SVP first randomly selects a subset $\Omega$ of the state-action pairs. In particular, each state-action pair in $\mathcal{S}\times\mathcal{A}$ is observed (i.e., included in $\Omega$) independently with probability $p$. 
- For each selected $(s,a)$, the intermediate $\hat{Q}(s,a)$ is computed based on the $Q$-value iteration: 
    \begin{equation*}\hat{Q}(s,a) \leftarrow \sum_{s'} P(s'|s,a) \left( r(s,a) + \gamma \max_{a'} Q^{(t)}(s',a') \right),\quad\forall\:(s,a)\in\Omega.
    				\end{equation*}
- The current iteration then ends by reconstructing the full $Q$ matrix with matrix estimation, from the set of observations in $\Omega$. That is, $Q^{(t+1)}=\textrm{ME}\big(\{\hat{Q}(s,a)\}_{(s,a)\in\Omega}\big).$

Overall, each iteration reduces the computation cost by roughly $1-p$.

Through SVP, we can compute the final state-action $Q$-value function.
To obtain the optimal policy for state $s$, we compute

\begin{align*}
    \pi^{\star} \left(s\right) = \mbox{argmax}_{a \in \mathcal{A}} Q^{\star}\left(s, a\right).
\end{align*}

### Generate state-action value function with SVP

In [1]:
using Pkg
Pkg.add("IJulia")
Pkg.add("PGFPlots")
Pkg.add("GridInterpolations")
Pkg.add("PyCall")
Pkg.add("ImageMagick")
Pkg.add("TickTock")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
push!(LOAD_PATH, ".")
using MDPs, CartPole, Printf, LinearAlgebra
mdp = MDP(state_space(), action_space(), transition, reward)
__init__()
policy = value_iteration(mdp, true, "../data/qcp_otf_0.2.csv", true)
print("")  # suppress output

Starting value iteration...
Iteration 1: average residual = 1.90e-01, cputime = 1.28e+02 sec
1Iteration 2: average residual = 1.46e-01, cputime = 1.34e+02 sec
2Iteration 3: average residual = 1.11e-01, cputime = 1.23e+02 sec
3Iteration 4: average residual = 1.07e-01, cputime = 1.20e+02 sec
4Iteration 5: average residual = 9.43e-02, cputime = 1.20e+02 sec
5Iteration 6: average residual = 7.27e-02, cputime = 1.21e+02 sec
6Iteration 7: average residual = 5.95e-02, cputime = 1.22e+02 sec
7Iteration 8: average residual = 5.41e-02, cputime = 1.27e+02 sec
8Iteration 9: average residual = 5.18e-02, cputime = 1.22e+02 sec
9Iteration 10: average residual = 5.07e-02, cputime = 1.28e+02 sec
Maximum number of iterations reached!
10Value iteration took 0 iterations and 1.25e+03 sec


┌ Info: Precompiling MDPs [top-level]
└ @ Base loading.jl:1423
┌ Info: Precompiling CartPole [top-level]
└ @ Base loading.jl:1423
┌ Info:  started timer at: 2021-12-09T16:16:03.288
└ @ TickTock /home/tsam/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/tsam/.julia/packages/PyCall/3fwVL/src/numpy.jl:67
┌ Info:  started timer at: 2021-12-09T16:18:11.651
└ @ TickTock /home/tsam/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2021-12-09T16:20:26.069
└ @ TickTock /home/tsam/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2021-12-09T16:22:29.278
└ @ TickTock /home/tsam/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2021-12-09T16:24:29.129
└ @ TickTock /home/tsam/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2021-12-09T16:2

In [3]:
policy

Policy([7.446926851223992e-13 7.440947286081739e-13 … 6.918198991673084e-13 8.208858930970773e-13; 0.6336192319286444 0.6329966421100621 … 0.6245639060533054 0.6240180950121222; … ; 0.6287892646786948 0.628188627764818 … 0.6309553152750879 0.6305133750666149; 3.621945353443917e-19 2.006819086368288e-19 … 5.35307410704286e-20 8.200325053599388e-20], [-10.0, -9.97997997997998, -9.95995995995996, -9.93993993993994, -9.91991991991992, -9.8998998998999, -9.87987987987988, -9.85985985985986, -9.83983983983984, -9.81981981981982  …  9.81981981981982, 9.83983983983984, 9.85985985985986, 9.87987987987988, 9.8998998998999, 9.91991991991992, 9.93993993993994, 9.95995995995996, 9.97997997997998, 10.0], 1000, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

### Visualize policy as a heat map

Note that the cart-pole problem is a 4-dimensional task, and the policy heat map should have 4 dims (but hard to visualize). Since the metric is the angle deviation, we here only plot the first two dims (i.e., the `angle` and `angular speed`) to visualize the heat map.

In [6]:
viz_policy(mdp, policy, "SVP policy (20% observed)", true, "cp/policy_cp_0.2.tex")

### Verify correctness
Simulate and visualize trajectory from initial state `[angle, angular_speed, position, x_speed]`.

In [7]:
ss, as = simulate(mdp, policy, [-0.5, 0.0, 0.0, 0.0])
viz_trajectory(ss, as, "SVP trajectory (20%)", "SVP input (20%)", true, "cp/traj_cp_0.2.tex")


In [9]:
nsim = 2000
deviation = 0
for sim = 1:nsim
    state = [rand(THETAMIN:0.001 * (THETAMAX - THETAMIN):THETAMAX), 
             rand(THETADOTMIN:0.001 * (THETADOTMAX - THETADOTMIN):THETADOTMAX),
             rand(XMIN:0.001 * (XMAX - XMIN):XMAX), 
             rand(XDOTMIN:0.001 * (XDOTMAX - XDOTMIN):XDOTMAX)]
    traj, _ = simulate(mdp, policy, copy(state))
    deviation += norm(traj[51:end, 1])
end # for sim
deviation /= nsim
@printf("average deviation: %.3f\n", deviation)

average deviation: 10.501
